# How to visualize data from Databricks

## Install dependencies

In [0]:
!pip install pydeck-carto pydeck carto-auth -q

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9dc8dd28-5e51-4a73-ad30-d9e6ef05894c/bin/python -m pip install --upgrade pip' command.


In [0]:
import pydeck as pdk
from pydeck_carto import register_carto_layer, get_layer_credentials
from pydeck_carto.layer import MapType, CartoConnection
from pydeck_carto.styles import color_bins
from carto_auth import CartoAuth

## Authenticate with CARTO

The method below will get an OAuth access token that will give you permission to interact with the CARTO platform with the same privileges and access to resources as if you were logged in.

In [0]:
carto_auth = CartoAuth.from_oauth()

Go to the following URL:
https://auth.carto.com/authorize?client_id=AtxvHDeuXlR8XPfF2nj2Uv2I29pvmCxu&response_type=code&scope=None&audience=carto-cloud-native-api&redirect_uri=https%3A%2F%2Fcallbacks.carto.com%2Ftoken&code_challenge_method=S256&code_challenge=QbSsrvk-qECBhGSopyOW4sl99jA1yS90g7Rj21t2HWU
Enter the Access code: 


 G_NwnWDS1NIitlWpUM4X-Qicrel5hLci5jS9pVpyQXV3v

## Loading an H3 Layer from a Databricks connection.

CARTO can connect to a Databricks SQL Warehouse or a Databricks cluster to push down SQL queries that will be fully executed in your Databricks environment. In both cases, the connection fully supports H3 indexes, leveraging [Databricks' native capabilities](https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-functions-builtin.html#h3-geospatial-functions).

By using the examples in this notebook, you're leveraging CARTO APIs from within your Databricks Workspace.

Learn more about how to connect CARTO to Databricks [here](https://docs.carto.com/carto-user-manual/connections/creating-a-connection/#connection-to-databricks).

In [0]:
# Register CartoLayer in pydeck
register_carto_layer()

# Render CartoLayer in pydeck
layer = pdk.Layer(
    "CartoLayer",
    data="hive_metastore.carto_dev_data.derived_spatialfeatures_ukr_h3int_res10_v1_yearly_v2_interpolated",
    type_=MapType.TABLE,
    connection=pdk.types.String("databricksconn_cluster"),
    credentials=get_layer_credentials(carto_auth),
    aggregation_exp=pdk.types.String("sum(population) as population"),
    aggregation_res_level=5,
    geo_column=pdk.types.String("h3"),
    get_fill_color=color_bins("population", [1, 10, 100, 1000, 10000, 100000], "SunsetDark"),
    get_line_color=[0, 0, 0, 80],
    line_width_min_pixels=0.5,
    stroked=True,
    extruded=False,
    pickable=True
)

tooltip = {
    "html": "Population: <b>{population}</b>",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

view_state = pdk.ViewState(latitude=49.0, longitude=30.0, zoom=5)
pdk.Deck(layer, map_style=pdk.map_styles.ROAD, initial_view_state=view_state)


<iframe
 width="100%"
 height=500
 frameborder="0"
 srcdoc="<!DOCTYPE html>
<html>
 <head>
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 <title>pydeck</title>
 <script src="https://api.tiles.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" />
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" />
 <script src='https://cdn.jsdelivr.net/npm/@deck.gl/jupyter-widget@~8.8.12/dist/index.js'></script>
 <style>
 body {
 margin: 0;
 padding: 0;
 overflow: hidden;
}

#deck-container {
 width: 100vw;
 height: 100vh;
}

#deck-container canvas {
 z-index: 1;
 background: none;
}
 </style>
 </head>
 <body>
 <div id="deck-container">
 </div>
 </body>
 <script>
 const container = document.getElementById('deck-container');
 const jsonInput = {
 "initialViewState": {
 "latitude": 49.0,
 "longitude": 30.0,
 "zoom": 5
 },
 "layers": [
 {
 "@@type": "CartoLayer",
 "aggregationExp": "sum(population) as population",
 "aggregationResLevel": 5,
 "clientId": "pydeck-carto",
 "connection": "databricksconn_cluster",
 "credentials": {
 "accessToken": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRVNGNZTHAwaThjYnVMNkd0LTE0diJ9.eyJodHRwOi8vYXBwLmNhcnRvLmNvbS9lbWFpbCI6ImFmYmxhbmNoYXJ0K3BtQGNhcnRvLmNvbSIsImh0dHA6Ly9hcHAuY2FydG8uY29tL2FjY291bnRfaWQiOiJhY19qZmpqb2Y1bSIsImlzcyI6Imh0dHBzOi8vYXV0aC5jYXJ0by5jb20vIiwic3ViIjoiYXV0aDB8NjI3MjNkODcyZWIxNzMwMDY4MTYwZjBlIiwiYXVkIjoiY2FydG8tY2xvdWQtbmF0aXZlLWFwaSIsImlhdCI6MTY2NTY2ODU1OSwiZXhwIjoxNjY1NzU0OTU5LCJhenAiOiJBdHh2SERldVhsUjhYUGZGMm5qMlV2MkkyOXB2bUN4dSIsInBlcm1pc3Npb25zIjpbImFkbWluOmFjY291bnQiLCJyZWFkOmFjY291bnQiLCJyZWFkOmFwcHMiLCJyZWFkOmNvbm5lY3Rpb25zIiwicmVhZDpjdXJyZW50X3VzZXIiLCJyZWFkOmltcG9ydHMiLCJyZWFkOmxpc3RlZF9hcHBzIiwicmVhZDptYXBzIiwicmVhZDp0aWxlc2V0cyIsInJlYWQ6dG9rZW5zIiwidXBkYXRlOmN1cnJlbnRfdXNlciIsIndyaXRlOmFwcHMiLCJ3cml0ZTpjb25uZWN0aW9ucyIsIndyaXRlOmltcG9ydHMiLCJ3cml0ZTpsaXN0ZWRfYXBwcyIsIndyaXRlOm1hcHMiLCJ3cml0ZTp0b2tlbnMiXX0.EqJ9ACAbzvB0_cTLzaVeGjDQHEyTA2wuffERLlB5E-ySkdbOAhgIEzrPh-mHJRAsVq_7xR_p2m8c-J4ukKE6Xz1vbKImpo236j4Dxgl_pEqh9heuG-3qE3487O90M-oz5T9U2IyWFLFTcJzSpKvWyNqIm1acSWE8A3W6e47wJcDp0b03cIMY0selold4B8OAxGUuwWNlY-IyNW6V1vYz-XGhPNISP_HB_c8SC4C3BVs9vgDfO71H2AmVFCwdB_xNzJLlU_bzjhu1TnL9M_oeLtluSfYUqHueull6PlNiYmior9Cpa7qtD0dwpabng7G8jJsw5N3jmOJI-S2HJ8inng",
 "apiBaseUrl": "https://gcp-us-east1.api.carto.com",
 "apiVersion": "v3"
 },
 "data": "hive_metastore.carto_dev_data.derived_spatialfeatures_ukr_h3int_res10_v1_yearly_v2_interpolated",
 "extruded": false,
 "geoColumn": "h3",
 "getFillColor": {
 "@@function": "colorBins",
 "attr": "population",
 "colors": "SunsetDark",
 "domain": [
 1,
 10,
 100,
 1000,
 10000,
 100000
 ],
 "nullColor": [
 204,
 204,
 204
 ]
 },
 "getLineColor": [
 0,
 0,
 0,
 80
 ],
 "id": "a45a5bcc-6431-4665-af37-1f5461265d04",
 "lineWidthMinPixels": 0.5,
 "onDataError": {
 "@@function": "notifyError"
 },
 "pickable": true,
 "stroked": true,
 "type": "table"
 }
 ],
 "mapProvider": "carto",
 "mapStyle": "https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json",
 "views": [
 {
 "@@type": "MapView",
 "controller": true
 }
 ]
};
 const tooltip = true;
 const customLibraries = [{'libraryName': 'CartoLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr.net/combine/npm/h3-js@~3.7.*/dist/h3-js.umd.js,npm/@deck.gl/extensions@~8.8.12/dist.min.js,npm/@deck.gl/carto@~8.8.12/dist.min.js'}];
 const configuration = {
 functions: {
 notifyError: () => {
 return (e) => {
 document.body.style.cssText = 'padding:24px;font-size:14px;font-family:monospace';
 document.body.innerHTML = `<b>Layer Error</b>: ${e.message}`;
 }
 }
 }
 };

 const deckInstance = createDeck({
 container,
 jsonInput,
 tooltip,
 customLibraries,
 configuration
 });

 </script>
</html>"
 >